# Methods benchmark on Binary MNIST 

In [1]:
import sys

import numpy as np

sys.path.append('..')
import torch

from relu_nets import ReLUNet
import neural_nets.data_loaders as data_loaders
import neural_nets.train as train

from hyperbox import Hyperbox
from lipMIP import LipMIP
from other_methods import LipSDP, StochasticApproximation, StochasticApproximationUCBDynamic 

In [17]:
input_dimension = 784
network_MNIST = ReLUNet([input_dimension, 4, 2]) # simple MNIST network 

mnist_train = data_loaders.load_mnist_data('train', digits=[1, 7], batch_size=64, shuffle=True) # Training data
mnist_val = data_loaders.load_mnist_data('val', digits=[1, 7], batch_size=64, shuffle=True) # Validation data 

vanilla_train_params = train.TrainParameters(mnist_train, mnist_val, 5, 
                                             test_after_epoch=1,
                                             optimizer = torch.optim.Adam(network_MNIST.parameters(), lr=0.001)) 

train.training_loop(network_MNIST, vanilla_train_params)

In [18]:
c_vector = torch.tensor([1.0, -1.0])
domain = Hyperbox.build_unit_hypercube(input_dimension)

In [19]:
res_dict = {"LipMIP": [], "LipSDP": [], "SA": [], "SA_UCB": []}
time_dict = {"LipMIP": [], "LipSDP": [], "SA": [], "SA_UCB": []}

simple_prob = LipMIP(network=network_MNIST, domain=domain, c_vector=c_vector, verbose=False, num_threads=5, timeout=300)
simple_result = simple_prob.compute_max_lipschitz()
res_lipmip = simple_result.as_dict()['value']
res_dict["LipMIP"].append(res_lipmip)
time_dict["LipMIP"].append(simple_result.as_dict()['compute_time'])
print(res_lipmip)

In [20]:
for i in range(5):
    lipSdp = LipSDP(network=network_MNIST, c_vector=c_vector, domain=domain)
    res_sdp = lipSdp.compute() * np.sqrt(input_dimension) # Because LipSDP overestimates L2 norm, scaling with sqrt(input_dim) needed 
    res_dict["LipSDP"].append(res_sdp)
    time_dict["LipSDP"].append(lipSdp.compute_time)
    print(res_sdp)
    
    SA = StochasticApproximation(network=network_MNIST, c_vector=c_vector, domain=domain, use_c_vector=True)
    res_sa = SA.compute(v=False, max_iter=5000, exact=None, tol=1, mode='Relative')
    res_dict["SA"].append(res_sa)
    time_dict["SA"].append(SA.compute_time)
    print(res_sa)
        
    SA_UCB = StochasticApproximationUCBDynamic(network=network_MNIST, c_vector=c_vector, domain=domain, c=1, partition_step=2, use_c_vector=True)
    res_sa_ucb = SA_UCB.compute(v=False, max_iter=5000, exact=None, tol=1, mode='Relative')
    res_dict["SA_UCB"].append(res_sa_ucb.item())
    time_dict["SA_UCB"].append(SA_UCB.compute_time)
    print(res_sa_ucb)

In [21]:
print(res_dict)
print(time_dict)

In [22]:
def f(x, orig):
    return np.abs(np.array(x) - orig)

def rel(x, orig):
    return (100 * (x - orig) / orig)

print(network_MNIST)
for key in res_dict.keys():
    if key == "LipMIP":
        print(f"{key}: {res_dict[key][0]} in {time_dict[key][0]}, (err = {0})")
        continue
    cur_best = 0
    for j in range(len(res_dict[key])):
        if rel(res_dict[key][j], res_dict["LipMIP"][0]) == rel(res_dict[key][cur_best], res_dict["LipMIP"][0]):
            if time_dict[key][j] < time_dict[key][cur_best]:
                cur_best = j
        if np.abs(rel(res_dict[key][j], res_dict["LipMIP"][0])) < np.abs(rel(res_dict[key][cur_best], res_dict["LipMIP"][0])):
            cur_best = j
    print(f"{key}: {res_dict[key][cur_best]} in {time_dict[key][cur_best]}, (err = {rel(res_dict[key][cur_best], res_dict['LipMIP'][0])})")
    